In [1]:
from ipyleaflet import Map, Marker, Icon, MarkerCluster, AwesomeIcon, Popup, basemaps, basemap_to_tiles
from ipywidgets import HTML

import pandas as pd
import map_methods
import math

In [2]:
hospital_data = pd.read_csv("../podatoci/AdditionalInfo-Hospitals.csv")

hospital_data['hospital'] = hospital_data.Hospital

In [3]:
sick = pd.read_csv("../podatoci/Корона вирус податоци Македонија - Хоспитализирани.csv", header=0,
                 names=['hospital', "date", "count", "age_range"])

recovered = pd.read_csv("../podatoci/Корона вирус податоци Македонија - Оздравени по Болница.csv", header=0,
                 names=['hospital', "date", "count", "age_range"])

dead = pd.read_csv("../podatoci/Корона вирус податоци Македонија - Мртви по Болница.csv", header=0,
                 names=['hospital', "date", "count", "age_range"])


sick_hospital = sick.groupby(["hospital"]).sum()
recovered_hospital = recovered.groupby(["hospital"]).sum()
dead_hospital = dead.groupby(["hospital"]).sum()

dead

,hospital,date,count,age_range


In [4]:
merged = pd.merge(sick_hospital, recovered_hospital, on='hospital', how='outer')
merged = pd.merge(merged, dead_hospital, on='hospital', how='outer')
merged.fillna(0, inplace=True)
merged = pd.merge(merged, hospital_data, on='hospital', how='inner')
merged = merged.set_index("hospital")
merged

,count_x,count_y,date,count,age_range,Hospital,Latitude,Longitude
hospital,,,,,,,,
Клиника за инфективни болести Скопје,14,1.0,0.0,0.0,0.0,Клиника за инфективни болести Скопје,41.988005,21.425646
Општа болница Дебар,5,0.0,0.0,0.0,0.0,Општа болница Дебар,41.523717,20.523101


In [5]:
center=(41.6086, 21.7453)

m = map_methods.get_macedonia_map()

def hide_marker(name, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    return (infected - healed - dead) < 1

def get_marker(name, location, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    current = infected - healed - dead
    scale = math.log(current, 2)
    hospital_icon = Icon(
        icon_url='https://www.freeiconspng.com/uploads/ambulance-cross-hospital-icon-11.png',
        icon_size=(10*scale, 10*scale),
        icon_anchor=(5*scale, 5*scale),
        popup_anchor=(0, -5*scale)
    )
    return Marker(location=location, draggable=False, icon=hospital_icon, z_index_offset=1000)

map_methods.add_marker_layer(m=m, d=merged, 
                             cols=["count_x", "count_y", "count"], 
                             names=["Заразени", "Излечени" ,"Мртви"],
                            get_marker=get_marker,
                            hide_marker=hide_marker,
                            lng='Longitude',
                            lat='Latitude')

m

Map(center=[41.6086, 21.7453], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [6]:
text

NameError: name 'text' is not defined